In [1]:
import sys
import ok
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import PIL.Image as img
import numpy as np
import cv2
from IPython.display import clear_output
%matplotlib tk

In [2]:
dev = ok.okCFrontPanel()
deviceCount = dev.GetDeviceCount()
for i in range(deviceCount):
    print 'Device[{0}] Model: {1}'.format(i, dev.GetDeviceListModel(i))
    print 'Device[{0}] Serial: {1}'.format(i, dev.GetDeviceListSerial(i))

Device[0] Model: 21
Device[0] Serial: 143400091C


In [3]:
dev = ok.okCFrontPanel()
dev.OpenBySerial("")
error = dev.ConfigureFPGA("cntr_fifo_str0.bit")
# Its a good idea to check for errors here

In [4]:
# IsFrontPanelEnabled returns true if FrontPanel is detected.
if True == dev.IsFrontPanelEnabled():
    print "FrontPanel host interface enabled."
else:
    sys.stderr.write("FrontPanel host interface not detected.")

FrontPanel host interface enabled.


In [5]:
row = 16
col = 16
count = 1
im_int = range(32)
datain128 = bytearray(131072)
datain1 = bytearray(51200)

In [6]:
#im = np.array(img.open('imTest.jpg'))
#imgplot = plt.imshow(im)
im = np.zeros(shape=(row ,col))
#plt.figure(3)
#plt.imshow(im).set_cmap('gray')
#plt.show()

In [7]:
# assert reset signal to initialize the FIFO.
dev.SetWireInValue(0x10, 0xff, 0x01);
dev.UpdateWireIns();
# deactivate reset signal and activate counter.
dev.SetWireInValue(0x10, 0x00, 0x01);
dev.UpdateWireIns();   

In [10]:
# Trigger the counter
dev.ActivateTriggerIn(0x53, 0x01)
cv2.namedWindow("imFull",cv2.WINDOW_NORMAL)
cv2.namedWindow("im",cv2.WINDOW_NORMAL)
while True:
    # Check for FIFO flag
    dev.UpdateTriggerOuts()
    # If the FIFO is full, read everything and display one frame only and exit the while loop
    if dev.IsTriggered(0x6A, 0x01) == True:
        # print 'FIFO full! ', count, ' times'
        count = count + 1
        dev.ReadFromPipeOut(0xA0, datain128)
        for i in range(row):
            im[i] = datain128[i*col:i*col+col]
        im = im/255
        cv2.imshow('imFull',im)
        cv2.waitKey(1)
        dev.ActivateTriggerIn(0x53, 0x02)
        # break
    # If one frame is ready in FIFO
    if dev.IsTriggered(0x6A, 0x04) == True:
        dev.ReadFromPipeOut(0xA0, datain1)
        for i in range(row):
            im[i] = datain1[i*col:i*col+col]
        im = im/255
        cv2.waitKey(1)
        cv2.imshow('im',im)

KeyboardInterrupt: 

In [9]:
print im*255

[[ 239.  238.  237.  236.  243.  242.  241.  240.  247.  246.  245.  244.
   251.  250.  249.  248.]
 [ 255.  254.  253.  252.    3.    2.    1.    0.    7.    6.    5.    4.
    11.   10.    9.    8.]
 [  15.   14.   13.   12.   19.   18.   17.   16.   23.   22.   21.   20.
    27.   26.   25.   24.]
 [  31.   30.   29.   28.   35.   34.   33.   32.   39.   38.   37.   36.
    43.   42.   41.   40.]
 [  47.   46.   45.   44.   51.   50.   49.   48.   55.   54.   53.   52.
    59.   58.   57.   56.]
 [  63.   62.   61.   60.   67.   66.   65.   64.   71.   70.   69.   68.
    75.   74.   73.   72.]
 [  79.   78.   77.   76.   83.   82.   81.   80.   87.   86.   85.   84.
    91.   90.   89.   88.]
 [  95.   94.   93.   92.   99.   98.   97.   96.  103.  102.  101.  100.
   107.  106.  105.  104.]
 [ 111.  110.  109.  108.  115.  114.  113.  112.  119.  118.  117.  116.
   123.  122.  121.  120.]
 [ 127.  126.  125.  124.  131.  130.  129.  128.  135.  134.  133.  132.
   139.  138.  13